In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import PolynomialFeatures
from math import sqrt

df1 = pd.read_csv("train.csv")
df2 = pd.read_csv("test.csv")

df1 = df1.drop(columns=['sub_area'])
df2 = df2.drop(columns=['sub_area', 'row ID'])

In [18]:
df1_encoded = pd.get_dummies(df1, drop_first=True)
df1_encoded.drop_duplicates(inplace=True)
df2_encoded = pd.get_dummies(df2, drop_first=True)

X = df1_encoded.drop(columns=['price_doc'], axis=1)
y = df1_encoded['price_doc']

# imputer = SimpleImputer(strategy='mean')
# X = imputer.fit_transform(X)
# df2_encoded = imputer.fit_transform(df2_encoded)

# scaler = MinMaxScaler()
# X_scaled = scaler.fit_transform(X)
# df2_encoded_scaled = scaler.fit_transform(df2_encoded)

In [19]:
variance_threshold = 0.001  # Set your desired threshold
selector = VarianceThreshold(threshold=variance_threshold)
X_train_high_variance = selector.fit_transform(X)
X_test_high_variance = selector.transform(df2_encoded) 


In [20]:
# Set the number of principal components
pca = PCA(n_components=15) # 15 is hbest
X_train_pca = pca.fit_transform(X_train_high_variance)
X_test_pca = pca.transform(X_test_high_variance)


In [21]:
poly_features = PolynomialFeatures(degree=3, include_bias=False)
X_train_poly = poly_features.fit_transform(X_train_pca)
X_test_poly = poly_features.transform(X_test_pca)


In [22]:
# Use Ridge regression
alpha = 0.001  # Set your desired regularization strength
ridge_model = Ridge(alpha=alpha)
ridge_model.fit(X_train_poly, y)


c:\Users\Irtiza\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=4.00806e-41): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


Ridge(alpha=0.001)

In [23]:
 # Make predictions on the training set for evaluation (you can adjust this part as needed)
pred_train = ridge_model.predict(X_train_poly)

# Print RMSE on the training set for evaluation
rmse_train = sqrt(mean_squared_error(y, pred_train))
print("RMSE on Training Set:", rmse_train)

# Generate predictions for the test set
pred_test = ridge_model.predict(X_test_poly)

print(pred_test)

df_sample = pd.read_csv("sample_submission.csv")
df_sample['price_doc'] = pred_test
df_sample.to_csv("predictions_ridge_25nov.csv", index=False)

RMSE on Training Set: 13255169.408429138
[6455639.80268568 8866352.69923679 6725894.464556   ... 4445345.29133024
 4445345.29133024 4445345.29133024]
